https://github.com/isakbosman/CS271P/blob/main/nbs/CS274P_Lab_3_Neural%20Network%20.ipynb

# Environment

In [ ]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [ ]:
# %cd /content/gdrive/MyDrive/cs271p/data/

In [ ]:
# !pip install wandb
!pip install openpyxl


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from PIL import Image
import glob
from tqdm import tqdm
from tensorflow import keras
from tensorflow.keras.layers import Dense, Conv3D, Flatten, Dropout, MaxPooling3D,Input,Conv2D,MaxPooling2D,GlobalAveragePooling2D
from tensorflow.keras.preprocessing import image
from tensorflow.keras.mixed_precision import experimental as mixed_precision


# import wandb


In [ ]:
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_policy(policy)

# Data Prepare

In [ ]:
train_df = pd.read_excel('../input/cs273p-data/train-pairs-updated.xlsx')
val_df = pd.read_excel('../input/cs273p-data/val-pairs-updated.xlsx')
val_df

In [ ]:
train_df = train_df[['p1','p2','ptype']]
val_df = val_df[['p1','p2','ptype']]

In [ ]:
train_df = train_df.loc[train_df["ptype"] != 'sibs' ].reset_index(drop=True)
val_df = val_df.loc[val_df["ptype"] != 'sib' ].reset_index(drop=True)

In [ ]:
val_df

In [ ]:
train_df.groupby(by=["ptype"]).count()

In [ ]:
label = pd.unique(train_df['ptype'])
label

In [ ]:
pd.unique(val_df['ptype'])

In [ ]:
label_to_index = dict((name, index) for index, name in enumerate(label))
label_to_index

In [ ]:
# label_to_index["sib"] = 6

In [ ]:
train_length = len(train_df)
val_length = len(val_df)

In [ ]:
train_list1 = []
train_list2 = []
train_label = []
val_list1 = []
val_list2 = []
val_label = []

In [ ]:
for num in tqdm(range(train_length)):
    train_path1 = "../input/cs273p-data/train-faces/train-faces/"+train_df["p1"][num]+"/*.jpg"
    train_path2 = "../input/cs273p-data/train-faces/train-faces/"+train_df["p2"][num]+"/*.jpg"
    for filename1 in glob.glob(train_path1):
        for filename2 in glob.glob(train_path2):
            train_list1.append(filename1)
            train_list2.append(filename2)
            train_label.append(label_to_index.get(train_df["ptype"][num]))

In [ ]:
train_label[90]

In [ ]:
for num in tqdm(range(val_length)):
    val_path1 = "../input/cs273p-data/val-faces/val-faces/"+val_df["p1"][num]+"/*.jpg"
    val_path2 = "../input/cs273p-data/val-faces/val-faces/"+val_df["p2"][num]+"/*.jpg"
    for filename1 in glob.glob(val_path1):
        for filename2 in glob.glob(val_path2):
            val_list1.append(filename1)
            val_list2.append(filename2)
            val_label.append(label_to_index.get(val_df["ptype"][num]))

In [ ]:
val_label[0]

In [ ]:
df_label = pd.DataFrame(train_label, columns=['label'])
df_label = df_label.groupby(by=['label']).size()
df_label

https://keras.io/examples/vision/siamese_network/#putting-everything-together

https://colab.research.google.com/github/keras-team/keras-io/blob/master/examples/vision/ipynb/siamese_network.ipynb#scrollTo=KSU61vgnB7Z6

In [ ]:
def preprocess_image(filename):
    """
    Load the specified file as a JPEG image, preprocess it and
    resize it to the target shape.
    """
    image_string = tf.io.read_file(filename)
    image = tf.io.decode_jpeg(image_string,channels=1)
#     image = tf.image.convert_image_dtype(image,tf.float32)
    image = tf.image.resize(image,[108,124])
    image /= 255.0
    # image = tf.image.resize_with_pad(image,224,224)
    return image


def preprocess(anchor,positive,label):
    """
    Given the filenames corresponding to the three images, load and
    preprocess them.
    """

    return (tf.concat([preprocess_image(anchor),
        preprocess_image(positive)],2),label
    )


In [ ]:
BATCH_SIZE = 128

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices((train_list1,train_list2,train_label))
val_dataset = tf.data.Dataset.from_tensor_slices((val_list1,val_list2,val_label))

In [ ]:
train_dataset = train_dataset.shuffle(len(train_label),seed = 50)
val_dataset = val_dataset.shuffle(len(val_label),seed = 50)

In [ ]:
list(val_dataset.take(1).as_numpy_iterator())[0]

In [ ]:
train_dataset = train_dataset.map(preprocess)
val_dataset = val_dataset.map(preprocess)

In [ ]:
val_dataset

In [ ]:
# list(val_dataset.take(1).as_numpy_iterator())[0]

In [ ]:
train_dataset = train_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Base Line Model(InceptionResNetV2)

In [ ]:
# model = keras.Sequential([
#     # Block 1
#     Conv2D(64,(3,3), padding='same', activation='relu',use_bias=True, 
#            input_shape=(108,124,2)),
#     Conv2D(64,(3,3), padding='same', activation='relu'),
#     MaxPooling2D((2,2),strides=(2,2)),

#     # Block 2
#     Conv2D(128, (3,3), padding='same',activation='relu',),
#     Conv2D(128, (3,3), padding='same',activation='relu',),
#     MaxPooling2D((2,2),strides=(2,2)),

#     # Block 3
#     Conv2D(256, (3,3), padding='same',activation='relu',),
#     Conv2D(256, (3,3), padding='same',activation='relu',),
#     Conv2D(256, (3,3), padding='same',activation='relu',),
#     MaxPooling2D((2,2),strides=(2,2)),

#     # Block 4
#     Conv2D(512, (3,3), padding='same',activation='relu',),
#     Conv2D(512, (3,3), padding='same',activation='relu',),
#     Conv2D(512, (3,3), padding='same',activation='relu',),
#     MaxPooling2D((2,2),strides=(2,2)),

#     # Block 5
#     Conv2D(512, (3,3), padding='same',activation='relu',),
#     Conv2D(512, (3,3), padding='same',activation='relu',),
#     Conv2D(512, (3,3), padding='same',activation='relu',),
#     MaxPooling2D((2,2),strides=(2,2)),

#     # block 6
#     GlobalAveragePooling2D(),
#     Dense(1024,activation='relu'),
# #     Dropout(0.5),
#     Dense(1024,activation='relu'),
# #     Dropout(0.5),
#     Dense(10,activation='softmax', name='predictions')
# ])

In [ ]:
# model.load_weights('../input/cs273p-data/vgg_face_weights.h5')

In [ ]:
dummy_model = tf.keras.applications.InceptionResNetV2(
    include_top=False,
    weights=('imagenet'),
    input_shape=(108,124,3),
    classes=10,
    classifier_activation="softmax",
)

In [ ]:
def gray_weights(weights):
    for r in range(len(weights)):
        for c in range(len(weights[r])):
            weights[r][c] = np.average(weights[r][c], axis = 0)
    return weights

def get_model_len(model):
    num = 0
    for i,layer in enumerate(model.layers):
        num = max(num,i)
    return num+1

In [ ]:
model = tf.keras.applications.InceptionResNetV2(
    include_top=True,
    weights=None,
    input_shape=(108,124,2),
    classes=10,
    classifier_activation="softmax",
)

In [ ]:
for i,layer in enumerate(model.layers):
    if (i == 1):
        weights = dummy_model.get_layer(index=i).get_weights()[0]
#         bias = dummy_model.get_layer(index=i).get_weights()[1]
        weights = gray_weights(weights)
        layer.set_weights([weights[:,:,-2:,:]])
    if(i>1 and i < get_model_len(dummy_model)):
        if (dummy_model.get_layer(index=i).get_weights()!=[]):
            weights = dummy_model.get_layer(index=i).get_weights()
#             bias = dummy_model.get_layer(index=i).get_weights()[1]
            layer.set_weights(weights)

In [ ]:
out = Dense(10,activation='softmax',dtype='float32', name='predictions')(model.layers[-2].output)

In [ ]:
model = tf.keras.Model(model.input,out)

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
              metrics=['accuracy'])
# model.summary()


In [ ]:
early_stop = keras.callbacks.EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [ ]:
model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath="./InceptionResNetV2.h5",
    save_weights_only=False,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True)

In [ ]:
# wandb.init()

In [ ]:
history = model.fit(train_dataset,epochs=1000,validation_data=val_dataset,callbacks=[early_stop,model_checkpoint])

In [ ]:
def plot_model(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    epochs_range = range(len(history.history['accuracy']))
    plt.figure(figsize=(8, 8))
    plt.subplot(1, 2, 1)
    plt.plot(epochs_range, acc, label='Training Accuracy')
    plt.plot(epochs_range, val_acc, label='Validation Accuracy')
    plt.legend(loc='lower right')
    plt.title('Training and Validation Accuracy')
    plt.subplot(1, 2, 2)
    plt.plot(epochs_range, loss, label='Training Loss')
    plt.plot(epochs_range, val_loss, label='Validation Loss')
    plt.legend(loc='upper right')
    plt.title('Training and Validation Loss')
    plt.show()

In [ ]:
plot_model(history)

In [ ]:
predictions = model.predict(val_dataset)
pred = []
for x in predictions:
    pred.append(np.argmax(x))
confusion = tf.math.confusion_matrix(labels=val_label, predictions=pred)
print(confusion)

In [ ]:
# def preprocess(anchor,positive,label):
#     """
#     Given the filenames corresponding to the three images, load and
#     preprocess them.
#     """

#     return (tf.stack([preprocess_image(anchor),
#         preprocess_image(positive)],axis=2),label
#     )

In [ ]:
# train_dataset_3d = tf.data.Dataset.from_tensor_slices((train_list1,train_list2,train_label))
# val_dataset_3d = tf.data.Dataset.from_tensor_slices((val_list1,val_list2,val_label))
# train_dataset_3d = train_dataset_3d.shuffle(len(train_label),seed = 5)
# val_dataset_3d = val_dataset_3d.shuffle(len(val_label),seed = 5)
# train_dataset_3d = train_dataset_3d.map(preprocess)
# val_dataset_3d = val_dataset_3d.map(preprocess)
# train_dataset_3d = train_dataset_3d.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
# val_dataset_3d = val_dataset_3d.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)